<a href="https://colab.research.google.com/github/27ABH/AI-Data-Science-2025-26/blob/main/Poem_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [13]:
def generate_imagery_poem(topic):
    # These 'seeds' force the model to stay on topic for every single line
    line_starts = ["The leaves", "Gold turns", "Cold wind", "Falling"]
    poem_lines = []

    print(f"--- Generating Poem: {topic} ---")

    for start_phrase in line_starts:
        # We build a very specific prompt for each line
        prompt = f"Topic: {topic}\nDescription: {start_phrase}"
        input_ids = tokenizer.encode(prompt, return_tensors="pt")

        output_ids = model.generate(
            input_ids,
            max_new_tokens=8,       # Keep it very short to prevent rambling
            do_sample=True,
            temperature=0.7,
            top_k=50,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id,
            # Stop if the model tries to make a newline or a quote
            bad_words_ids=[tokenizer.encode(w) for w in ['"', "I ", " he", " says"]]
        )

        # Decode and clean up
        full_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        # Grab only the text generated AFTER our start_phrase
        line_content = full_text.split("Description: ")[-1].strip()

        # Ensure we only take the first sentence/fragment
        clean_line = line_content.split('\n')[0].split('.')[0]
        poem_lines.append(clean_line)

    # Final Output
    for i, line in enumerate(poem_lines):
        print(f"Line {i+1}: {line}")
    print("--- Done ---")

generate_imagery_poem("falling autumn leaves")

--- Generating Poem: falling autumn leaves ---
Line 1: The leaves are a wonderful combination of green and yellow
Line 2: Gold turns yellow when it falls, and red after
Line 3: Cold wind carries the scent of snow, and a
Line 4: Falling winter clouds with a heavy summer breeze
--- Done ---
